In [1]:
%load_ext autoreload
%autoreload 2

In [22]:
# 只需要修改get_activations.py，process_activations.py，create_NeuronRecord.py
#   中的activations_dir，就可以将激活值信息保存在其他路径
# model_dir是预训练模型的位置
!CUDA_VISIBLE_DEVICES=3 python get_activations.py

Now geting activations in neurons
max_length=64, folder_path='./ori_136000it', model_dir='/data/jqliu/ML_jq/nanoGPT/out_ori/out_test', MEAN_ACT=False
Loading dataset shards: 100%|██████████████████| 80/80 [00:00<00:00, 687.35it/s]
Now geting activations in neurons
max_length=64, folder_path='./ori_136000it', model_dir='/data/jqliu/ML_jq/nanoGPT/out_ori/out_test', MEAN_ACT=False
Now geting activations in neurons
max_length=64, folder_path='./ori_136000it', model_dir='/data/jqliu/ML_jq/nanoGPT/out_ori/out_test', MEAN_ACT=False
/home/jqliu/ML_jq/neuronExpainer/automated-interpretability/neuron-explainer/create_neuronRecord_and_score/get_activations.py:193: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more de

In [4]:
!CUDA_VISIBLE_DEVICES=9 python process_activations.py

Now processing activations of neurons
max_length=64, info_dict_path='./ori_136000it/info_dict_neurons_activations__len_64.json'
Loading dataset shards: 100%|███████████████████| 80/80 [01:14<00:00,  1.08it/s]
 those
 Clear
ist
 to
 by
 series
 the
 our
 of
 of
 a
 Motor
shard_0 finish!
tokenizing shard 1 (num_proc=8):  10%| | 12849/125216 [00:02<00:14, 7598.26 exam^C
tokenizing shard 1 (num_proc=8):  11%| | 13614/125216 [00:02<00:20, 5502.87 exam


In [8]:
!python create_NeuronRecord.py

create_from_info_dict_neurons
info_dict_path='./ori_136000it/info_dict_neurons_activations__len_64.json'
neuron_records_path='./ori_136000it/neuron_records_neurons.json'
layer_0, neuron_0 finished
layer_0, neuron_300 finished
layer_0, neuron_600 finished
layer_0, neuron_900 finished
layer_0, neuron_1200 finished
layer_0, neuron_1500 finished
layer_0, neuron_1800 finished
layer_0, neuron_2100 finished
layer_0, neuron_2400 finished
layer_0, neuron_2700 finished
layer_0, neuron_3000 finished
layer_1, neuron_0 finished
layer_1, neuron_300 finished
layer_1, neuron_600 finished
layer_1, neuron_900 finished
layer_1, neuron_1200 finished
layer_1, neuron_1500 finished
layer_1, neuron_1800 finished
layer_1, neuron_2100 finished
layer_1, neuron_2400 finished
layer_1, neuron_2700 finished
layer_1, neuron_3000 finished
layer_2, neuron_0 finished
layer_2, neuron_300 finished
layer_2, neuron_600 finished
layer_2, neuron_900 finished
layer_2, neuron_1200 finished
layer_2, neuron_1500 finished
layer_2,

In [19]:
!CUDA_VISIBLE_DEVICES=3 python generate_and_score.py

neuron_records_path='./ori_136000it/neuron_records_neurons.json'
random_neuron_indices=[2619, 456]
now evaluating feature_2619
!! now making request
!! making request done
Neuron 2619 explanation: terms and phrases related to aerospace technology and commercial space exploration.
Loading tokenizer and model...
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████████████| 4/4 [00:39<00:00,  9.86s/it]
Model and tokenizer loaded.
now using llama to simulate
/home/jqliu/miniconda3/envs/py3.9_autoInter/lib/python3.12/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
From v4.47 onwards, when a model cache is to b